In [60]:
#!pip3 install matplotlib
#!pip3 install sklearn
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
plt.style.use('ggplot')
#%matplotlib inline  

from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

from sklearn import model_selection #import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn import metrics  # mean_squared_error, mean_absolute_error, median_absolute_error, explained_variance_score, r2_score
from sklearn.feature_selection import SelectFromModel, RFECV
import Config 
import EDA
#binary classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn import metrics

from sklearn import model_selection



### Load Data:

In [61]:
# load training data prepared previously
# Load training data prepared previuosly
filepath =Config.data_wrangling
df = pd.read_csv(filepath)
df.fillna(0,inplace=True)
df.head()

,startDate,endDate,settings,lastCycleReached,id.engine.id,id.engine.maintenanceIndex,id.engine.lastTimeConnected,id.engine.description,id.engine.connected,id.cycle,...,sensorsMin.ax__max,sensorsMin.ay__max,sensorsMin.az__max,sensorsMin.temperature__max,sensorsMin.humidity__max,sensorsMin.temperatureIndex__max,lastCycle,labels.ttf,labels.bnc,labels.mcc
0,0,0,0,False,1,0,0,vehicula. Pellentesque tincidunt tempus risus....,False,0,...,-77.904986,-79.539219,-89.343668,0.0,0.0,0.0,4,4,0,0
1,0,0,0,False,1,0,0,vehicula. Pellentesque tincidunt tempus risus....,False,1,...,-88.504482,-89.364603,-86.998806,0.0,0.0,0.0,4,3,0,0
2,0,0,0,False,1,0,0,vehicula. Pellentesque tincidunt tempus risus....,False,2,...,-88.332153,-89.697292,-89.405550,0.0,0.0,0.0,4,2,0,0
3,0,0,0,False,1,0,0,vehicula. Pellentesque tincidunt tempus risus....,False,3,...,-75.930179,-87.669944,-88.780239,0.0,0.0,0.0,4,1,1,1
4,0,0,0,False,1,0,0,vehicula. Pellentesque tincidunt tempus risus....,False,4,...,-85.067708,-89.246504,-89.532829,0.0,0.0,0.0,4,0,1,2


### Regression Modelling:

Segment training and test data into features dataframe and labels series.  

To make it easy to train models on different set of features, a variable to hold the set of features required was used to subset the original dataframes

In [62]:
#Prepare data for regression model
features =EDA.L


X_df = df[features]
y_df = df['labels.ttf']

In [63]:
def get_regression_metrics(model, actual, predicted):
    
    """Calculate main regression metrics.
    
    Args:
        model (str): The model name identifier
        actual (series): Contains the test label values
        predicted (series): Contains the predicted values
        
    Returns:
        dataframe: The combined metrics in single dataframe
    
    
    """
    regr_metrics = {
                        'Root Mean Squared Error' : metrics.mean_squared_error(actual, predicted)**0.5,
                        'Mean Absolute Error' : metrics.mean_absolute_error(actual, predicted),
                        'R^2' : metrics.r2_score(actual, predicted),
                        'Explained Variance' : metrics.explained_variance_score(actual, predicted)
                   }

    #return reg_metrics
    df_regr_metrics = pd.DataFrame.from_dict(regr_metrics, orient='index')
    df_regr_metrics.columns = [model]
    return df_regr_metrics

Using the above functions let us model and evaluate some regression algorithms

In [115]:
ttf_models =[linear_model.LinearRegression(),linear_model.Lasso(alpha=0.001),linear_model.Ridge(alpha=0.01),DecisionTreeRegressor(max_depth=7, random_state=123),RandomForestRegressor(n_estimators=100, max_features=3, max_depth=4, n_jobs=-1, random_state=1)]
ttf_models_names =["LinearRegression","LASSO","RIDGE","DecisionTree","RandomForest"]

In [83]:
def consideredData(df):
    return df[df['lastCycleReached']==False]
def toPredictData(df):
    return df[df['lastCycleReached']==False]
def tryModelRegression(model,df,features,label):
    data = consideredData(df)
    X_df = data[features]
    Y_df = data[label]
    X_train,X_test,Y_train,Y_test =train_test_split(X_df,Y_df,test_size=0.33,random_state=42)
    trainedModel = model.fit(X_train,Y_train)
    Y_test_predicted = trainedModel.predict(X_test)
    modelScore = get_regression_metrics(trainedModel,Y_test,Y_test_predicted)
    return modelScore
    



In [84]:
score_ttf = pd.DataFrame()
for model in ttf_models :
    score_ttf = pd.concat([score_ttf,tryModelRegression(model,df,EDA.L,'labels.ttf')],axis=1)
score_ttf.columns = ttf_models_names
score_ttf.reset_index(inplace=True)
score_ttf

/home/mohamedelhedi-benyedder/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.001216514631141295, tolerance: 0.0004666666666666666
  model = cd_fast.enet_coordinate_descent(


,index,LogisticRegression,LASSO,RIDGE,DecisionTree,RandomForest
0,Root Mean Squared Error,2.689891,1.088956,1.698720,1.581139,1.699073
1,Mean Absolute Error,2.615927,1.082953,1.589606,1.500000,1.675000
2,R^2,-2.215783,0.472966,-0.282512,-0.111111,-0.283044
3,Explained Variance,-2.041365,0.478761,-0.123043,0.000000,-0.246944


In [85]:
def selectmodel(score,modelnames):
    m = score[modelnames].abs().mean().max()
    avg = score[modelnames].abs().mean()
    selectedModel = list(avg [avg == m].index)[0]
    return selectedModel


### Binary Classifcation:

In [86]:
bnc_models = [LogisticRegression(random_state=123),DecisionTreeClassifier(random_state=123),RandomForestClassifier(n_estimators=50, random_state=123),SVC(kernel='rbf', random_state=123),LinearSVC(random_state=123),GaussianNB()]
bnc_models_names=['LogisticRegression','DecisionTreeClassifier','RandomForestClassifier','SVC','LinearSVC','GaussianNB']

In [87]:
def bin_class_metrics(modelname, y_test, y_pred):
    
    """Calculate main binary classifcation metrics, plot AUC ROC and Precision-Recall curves.
    
    Args:
        model (str): The model name identifier
        y_test (series): Contains the test label values
        y_pred (series): Contains the predicted values
        y_score (series): Contains the predicted scores
        print_out (bool): Print the classification metrics and thresholds values
        plot_out (bool): Plot AUC ROC, Precision-Recall, and Threshold curves
        
    Returns:
        dataframe: The combined metrics in single dataframe
        
        
    """
      
    binclass_metrics = {
                        'Accuracy' : metrics.accuracy_score(y_test, y_pred),
                        'Precision' : metrics.precision_score(y_test, y_pred),
                        'Recall' : metrics.recall_score(y_test, y_pred),
                        'F1 Score' : metrics.f1_score(y_test, y_pred),
                       }

    df_metrics = pd.DataFrame.from_dict(binclass_metrics, orient='index')
    df_metrics.columns = [modelname]  


    

    return  df_metrics


In [88]:
def tryModelBNC(model,modelname,df,features,label,thresh=0.5):
    data = consideredData(df)
    X_df = df[features]
    Y_df = df[label]
    X_train,X_test,Y_train,Y_test =train_test_split(X_df,Y_df,test_size=0.33,random_state=42)
    trainedModel = model.fit(X_train,Y_train)
    Y_test_predicted = trainedModel.predict(X_test)
    Y_test_predicted[Y_test_predicted<thresh] = 0
    Y_test_predicted[Y_test_predicted>=thresh] = 1
    modelScore = bin_class_metrics(modelname, Y_test, Y_test_predicted)
    return modelScore

In [89]:
score_bnc = pd.DataFrame()
for i in range(len(bnc_models)) :

    score_bnc = pd.concat([score_bnc,tryModelBNC(bnc_models[i],bnc_models_names[i],df,EDA.L,'labels.bnc')],axis=1)
score_bnc.columns = bnc_models_names
score_bnc.reset_index(inplace=True)
score_bnc

/home/mohamedelhedi-benyedder/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mohamedelhedi-benyedder/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mohamedelhedi-benyedder/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/mohamedelhedi-benyedder/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Prec

,index,LogisticRegression,DecisionTreeClassifier,RandomForestClassifier,SVC,LinearSVC,GaussianNB
0,Accuracy,0.500000,0.5,0.0,0.5,0.500000,0.5
1,Precision,0.500000,0.0,0.0,0.0,0.500000,0.0
2,Recall,1.000000,0.0,0.0,0.0,1.000000,0.0
3,F1 Score,0.666667,0.0,0.0,0.0,0.666667,0.0


In [90]:
score_bnc.mean(axis=1).max()

0.4166666666666667

### Mutli Classifcation:

In [91]:
def multiclass_metrics(modelname, y_test, y_pred):
    
    """Calculate main multiclass classifcation metrics
    Args:
        model (str): The model name identifier
        y_test (series): Contains the test label values
        y_pred (series): Contains the predicted values
        
        
    Returns:
        dataframe: The combined metrics in single dataframe
        
  
    
    """
    multiclass_metrics = {
                            'Accuracy' : metrics.accuracy_score(y_test, y_pred),
                            'macro F1' : metrics.f1_score(y_test, y_pred, average='macro'),
                            'micro F1' : metrics.f1_score(y_test, y_pred, average='micro'),
                            'macro Precision' : metrics.precision_score(y_test, y_pred,  average='macro'),
                            'micro Precision' : metrics.precision_score(y_test, y_pred,  average='micro'),
                            'macro Recall' : metrics.recall_score(y_test, y_pred,  average='macro'),
                            'micro Recall' : metrics.recall_score(y_test, y_pred,average='macro'),
                        }
    
    df_metrics = pd.DataFrame.from_dict(multiclass_metrics, orient='index')
    df_metrics.columns = [model]

   
    
    return df_metrics

In [92]:
def tryModelMCC(model,modelname,df,features,label,thresh=0.5):
    data = consideredData(df)
    X_df = df[features]
    Y_df = df[label]
    X_train,X_test,Y_train,Y_test =train_test_split(X_df,Y_df,test_size=0.33,random_state=42)
    trainedModel = model.fit(X_train,Y_train)
    Y_test_predicted = trainedModel.predict(X_test)
    Y_test_predicted = np.round(Y_test_predicted)
    modelScore = multiclass_metrics(modelname, Y_test, Y_test_predicted)
    return modelScore 

In [93]:
score_mcc = pd.DataFrame()
for i in range(len(bnc_models)) :

    score_mcc = pd.concat([score_mcc,tryModelMCC(bnc_models[i],bnc_models_names[i],df,EDA.L,'labels.bnc')],axis=1)
score_mcc.columns = bnc_models_names
score_mcc.reset_index(inplace=True)
score_mcc

/home/mohamedelhedi-benyedder/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mohamedelhedi-benyedder/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mohamedelhedi-benyedder/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mohamedelhedi-benyedde

,index,LogisticRegression,DecisionTreeClassifier,RandomForestClassifier,SVC,LinearSVC,GaussianNB
0,Accuracy,0.500000,0.500000,0.0,0.500000,0.500000,0.500000
1,macro F1,0.333333,0.333333,0.0,0.333333,0.333333,0.333333
2,micro F1,0.500000,0.500000,0.0,0.500000,0.500000,0.500000
3,macro Precision,0.250000,0.250000,0.0,0.250000,0.250000,0.250000
4,micro Precision,0.500000,0.500000,0.0,0.500000,0.500000,0.500000
5,macro Recall,0.500000,0.500000,0.0,0.500000,0.500000,0.500000
6,micro Recall,0.500000,0.500000,0.0,0.500000,0.500000,0.500000


In [94]:
SelectedModels = [selectmodel(score_ttf,ttf_models_names),selectmodel(score_bnc,bnc_models_names),selectmodel(score_mcc,bnc_models_names)]
SelectedModels


['LogisticRegression', 'LogisticRegression', 'LogisticRegression']

In [95]:
SelectedModelsindexs = [ttf_models_names.index(SelectedModels[0]),bnc_models_names.index(SelectedModels[1]),bnc_models_names.index(SelectedModels[2])]
SelectedModelsindexs

[0, 0, 0]

In [98]:
def RunModels(models,df,features):
    datatrain = consideredData(df)
    dataToPredict = toPredictData(df)
    X_train = datatrain[features]
    #ttf
    Y_train_ttf = datatrain['labels.ttf']
    trainedModel_ttf = ttf_models[models[0]].fit(X_train,Y_train_ttf)
    dataToPredict['labels.ttf']= trainedModel_ttf.predict(dataToPredict[features])
    #bnc 
    Y_train_bnc = datatrain['labels.bnc']
    trainedModel_bnc = bnc_models[models[1]].fit(X_train,Y_train_bnc)
    dataToPredict['labels.bnc'] = trainedModel_bnc.predict(dataToPredict[features])
    #mcc 
    Y_train_mcc = datatrain['labels.mcc']
    trainedModel_mcc = bnc_models[models[2]].fit(X_train,Y_train_mcc)
    dataToPredict['labels.mcc']= trainedModel_mcc.predict(dataToPredict[features])
    return dataToPredict
    

In [114]:
cols = []
for k in df.columns.tolist():
    if (('id.' in k) or ('labels.' in k)):
        cols.append(k)
predictions =RunModels(SelectedModelsindexs,df,EDA.L)[cols]
predictions


/home/mohamedelhedi-benyedder/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,id.engine.id,id.engine.maintenanceIndex,id.engine.lastTimeConnected,id.engine.description,id.engine.connected,id.cycle,id.maintenanceIndex,labels.ttf,labels.bnc,labels.mcc
0,1,0,0,vehicula. Pellentesque tincidunt tempus risus....,False,0,0,4.000000e+00,0,0
1,1,0,0,vehicula. Pellentesque tincidunt tempus risus....,False,1,0,3.000000e+00,0,0
2,1,0,0,vehicula. Pellentesque tincidunt tempus risus....,False,2,0,2.000000e+00,0,0
3,1,0,0,vehicula. Pellentesque tincidunt tempus risus....,False,3,0,1.000000e+00,1,1
4,1,0,0,vehicula. Pellentesque tincidunt tempus risus....,False,4,0,-6.394885e-14,1,2
